In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
import copy
from sklearn.metrics import accuracy_score,f1_score,roc_auc_score, mean_absolute_error, mean_squared_error
from sklearn.feature_selection import RFE


import math
import numpy as np
import copy

import functions as func

LENGTH = 1000

labels = ["Prog1_scaled","Prog1_beste", "MfI1_beste", "beste"]
others = [["Prog1_beste","MfI1_beste","beste"],["MfI1_beste","beste","Prog1_scaled"],["Prog1_beste","beste","Prog1_scaled"],["Prog1_beste","MfI1_beste","Prog1_scaled"]]

labels_class = ["Prog1_beste", "MfI1_beste", "beste", "beide"]
others_class = [["MfI1_beste","beste","beide"],["Prog1_beste","beste","beide"],["Prog1_beste","MfI1_beste","beide"]
         ,["Prog1_beste","MfI1_beste","beste"]]


In [2]:
#Regression
estimator = RandomForestRegressor(n_estimators = 100, verbose=1, n_jobs=-1, random_state=42)

dt_reg = DecisionTreeRegressor(random_state=42)
#helper variables
predictions_rmse_train = [[0] * LENGTH for i in range(4)]
predictions_rmse_test = [[0] * LENGTH for i in range(4)]
predictions_mae_train = [[0] * LENGTH for i in range(4)]
predictions_mae_test = [[0] * LENGTH for i in range(4)]

rmse_reg_train = [[0] * 1 for i in range(4)]
rmse_reg_test = [[0] * 1 for i in range(4)]
mae_reg_train = [[0] * 1 for i in range(4)]
mae_reg_test = [[0] * 1 for i in range(4)]

selector_rmse = {}
selector_mae = {}

scenario = 0
other_count = 0
#Optimal number of attributes 
num_attributes_rmse = {"Prog1_scaled": 8,"Prog1_beste": 8, "MfI1_beste": 24, "beste": 27}
num_attributes_mae = {"Prog1_scaled": 8,"Prog1_beste": 7, "MfI1_beste": 4, "beste": 13}

for label in labels:   
    estimator = RandomForestRegressor(n_estimators = 100, verbose=1, n_jobs=-1, random_state=42)

    #Load data
    data,_,y,train_X,test_X,train_y,test_y,names,feature_names,numcols,numrows = func.load_data_noten(label,others[other_count][0],others[other_count][1],others[other_count][2])
    
    print("RMSE ",label)
    selector_rmse[label] = RFE(estimator, n_features_to_select= num_attributes_rmse[label], step=1, verbose=1)
    selector_rmse[label] = selector_rmse[label].fit(train_X, train_y)
    
    estimator = RandomForestRegressor(n_estimators = 100, verbose=1, n_jobs=-1, random_state=42)
    print("MAE ",label)
    selector_mae[label] = RFE(estimator, n_features_to_select=num_attributes_mae[label], step=1, verbose=1)
    selector_mae[label] = selector_mae[label].fit(train_X, train_y)
    
    #Select attributes
    #RMSE
    supp_rmse = selector_rmse[label].get_support() 
    sel_feat_rmse = train_X.columns[(supp_rmse)]
    sel_feat_rmse = sel_feat_rmse.tolist()
    print(len(sel_feat_rmse))
    print(sel_feat_rmse)
    train_X_rmse = copy.deepcopy(train_X[sel_feat_rmse])
    test_X_rmse = copy.deepcopy(test_X[sel_feat_rmse])
    #MAE
    supp_mae = selector_mae[label].get_support() 
    sel_feat_mae = train_X.columns[(supp_mae)]
    sel_feat_mae = sel_feat_mae.tolist()
    print(len(sel_feat_mae))
    print(sel_feat_mae)
    train_X_mae = copy.deepcopy(train_X[sel_feat_mae])
    test_X_mae = copy.deepcopy(test_X[sel_feat_mae])
    
    #Cross_Validation on train data
    func.cross_val(dt_reg, train_X_rmse, train_y, scoring="neg_mean_squared_error")
    func.cross_val(dt_reg, train_X_mae, train_y, scoring="neg_mean_absolute_error")
    
    
    #Fit and store predictions
    #RMSE
    dt_reg.fit(train_X_rmse, train_y)
    predictions_rmse_train[scenario] = dt_reg.predict(train_X_rmse)
    predictions_rmse_test[scenario] = dt_reg.predict(test_X_rmse)
    
    #MAE
    dt_reg.fit(train_X_mae, train_y)
    predictions_mae_train[scenario] = dt_reg.predict(train_X_mae)
    predictions_mae_test[scenario] = dt_reg.predict(test_X_mae)
    
    #Calculate performance
    rmse_reg_train[scenario] = sqrt(mean_squared_error(train_y,predictions_rmse_train[scenario]))
    rmse_reg_test[scenario] = sqrt(mean_squared_error(test_y,predictions_rmse_test[scenario]))

    mae_reg_train[scenario] = mean_absolute_error(train_y,predictions_mae_train[scenario])
    mae_reg_test[scenario] = mean_absolute_error(test_y,predictions_mae_test[scenario])
    
    
    scenario += 1
    other_count += 1



print("RMSE Trainingsdaten:", rmse_reg_train)
print("RMSE Testdaten:",rmse_reg_test)
print("MAE Trainingsdaten:",mae_reg_train)
print("MAE Testdaten:",mae_reg_test)




RMSE  Prog1_scaled
Fitting estimator with 84 features.


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.4s finished


Fitting estimator with 83 features.


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.2s finished


Fitting estimator with 82 features.


...

Fitting estimator with 15 features.


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.1s finished


Fitting estimator with 14 features.


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.1s finished


27
['Abinote', 'BFI_K_11', 'BFI_K_16', 'BFI_K_3', 'BFI_K_8', 'BM_Inf_14', 'BM_Inf_18', 'Ktyp_exp', 'Kurs_Inf', 'LMI_1', 'LMI_11', 'LMI_18', 'LMI_19', 'LMI_3', 'LMI_6', 'LMI_7', 'LMI_8', 'SKInf_1', 'SKInf_3', 'SKMat_1', 'mean_BFI_K_E', 'mean_BFI_K_G', 'mean_BFI_K_N', 'mean_BFI_K_V', 'mean_LMI', 'mean_SKInf', 'mean_SKMat']
13
['Abinote', 'BFI_K_11', 'BFI_K_8', 'LMI_3', 'LMI_7', 'LMI_8', 'SKMat_1', 'mean_BFI_K_G', 'mean_BFI_K_N', 'mean_BFI_K_V', 'mean_LMI', 'mean_SKInf', 'mean_SKMat']
Scores: [188.16250478 134.17363416 116.48704121 162.41597018 136.65168938]
Mean: 147.57816794005137
Standard deviation: 25.031885239272043
Scores: [-129.74358974 -101.28205128  -84.61538462 -132.63157895 -107.63157895]
Mean: -111.18083670715251
Standard deviation: 18.005646732764372
RMSE Trainingsdaten: [35.31829835868225, 35.318298358682, 0.0, 17.90880757060716]
RMSE Testdaten: [140.96949474473234, 163.2178306465438, 129.0348790056394, 175.85166909588753]
MAE Trainingsdaten: [9.581151832460804, 9.5811518324

In [3]:
#Classification
dt_class = DecisionTreeClassifier(random_state=42)
#helper variables
predictions_acc_train = [[0] * LENGTH for i in range(7)]
predictions_acc_test = [[0] * LENGTH for i in range(7)]
predictions_f1_train = [[0] * LENGTH for i in range(7)]
predictions_f1_test = [[0] * LENGTH for i in range(7)]
predictions_auc_train = [[0] * LENGTH for i in range(7)]
predictions_auc_test = [[0] * LENGTH for i in range(7)]

probs_class_train = [[0] * LENGTH for i in range(7)]
probs_class_test = [[0] * LENGTH for i in range(7)]


acc_train = [[0] * 1 for i in range(7)]
acc_test = [[0] * 1 for i in range(7)]
f1_train = [[0] * 1 for i in range(7)]
f1_test = [[0] * 1 for i in range(7)]
auc_train = [[0] * 1 for i in range(7)]
auc_test = [[0] * 1 for i in range(7)]



scenario = 0
other_count = 0

selector_acc = {}
selector_f1 = {}
selector_auc = {}
#Optimal number of attributes
num_attributes_acc_noten = {"Prog1_beste": 7, "MfI1_beste": 48, "beste": 6}
num_attributes_acc_bestanden = {"Prog1_beste": 6, "MfI1_beste": 35, "beste": 28, "beide": 59}
num_attributes_f1 = {"Prog1_beste": 6, "MfI1_beste": 35, "beste": 3}
num_attributes_auc = {"Prog1_beste": 5, "MfI1_beste": 66, "beste": 4}

#Classification for scenarios 1. - 3.
for label in labels:   
    #Continuuos labels after normalization and scaling not suitable for accuracy
    if label == "Prog1_scaled":
        other_count+=1
        continue
    #Load data
    data,_,y,train_X,test_X,train_y,test_y,names,feature_names,numcols,numrows = func.load_data_noten(label,others[other_count][0],others[other_count][1])
    
    print("Accuracy ",label)
    estimator = RandomForestClassifier(n_estimators = 100, verbose=1, n_jobs=-1, random_state=42)
    selector_acc[label] = RFE(estimator, num_attributes_acc_noten[label], step=1, verbose=1)
    selector_acc[label] = selector_acc[label].fit(train_X, train_y)
    #Select attributes
    supp_acc = selector_acc[label].get_support() 
    sel_feat_acc = train_X.columns[(supp_acc)]
    sel_feat_acc = sel_feat_acc.tolist()
    print(len(sel_feat_acc))
    print(sel_feat_acc)
    train_X_acc = copy.deepcopy(train_X[sel_feat_acc])
    test_X_acc = copy.deepcopy(test_X[sel_feat_acc])
    
    #Cross_Validation on train data
    func.cross_val(dt_class, train_X_acc, train_y, scoring="accuracy")
    
    #Fit and store predicitons
    dt_class.fit(train_X_acc, train_y)
    predictions_acc_train[scenario] = dt_class.predict(train_X_acc)
    predictions_acc_test[scenario] = dt_class.predict(test_X_acc)
    #Calculate accuracy
    acc_train[scenario] = accuracy_score(train_y,predictions_acc_train[scenario])
    acc_test[scenario] = accuracy_score(test_y,predictions_acc_test[scenario])   
    scenario += 1
    other_count += 1
    
other_count = 0
#Classification for scenarios 4. - 7.
for label in labels_class:   
    #Load data
    data,_,y,train_X,test_X,train_y,test_y,names,feature_names,numcols,numrows = func.load_data_bestanden(label,others_class[other_count][0],
                                                                 others_class[other_count][1],
                                                                 others_class[other_count][2])
    print("Accuracy ",label)
    estimator = RandomForestClassifier(n_estimators = 100, verbose=1, n_jobs=-1, random_state=42)
    selector_acc[label] = RFE(estimator, num_attributes_acc_bestanden[label], step=1, verbose=1)
    selector_acc[label] = selector_acc[label].fit(train_X, train_y)
    #Select attributes for accuracy
    supp_acc = selector_acc[label].get_support() 
    sel_feat_acc = train_X.columns[(supp_acc)]
    sel_feat_acc = sel_feat_acc.tolist()
    print(len(sel_feat_acc))
    print(sel_feat_acc)
    train_X_acc = copy.deepcopy(train_X[sel_feat_acc])
    test_X_acc = copy.deepcopy(test_X[sel_feat_acc])
    
    #Cross_Validation on train data for accuracy
    func.cross_val(dt_class, train_X_acc, train_y, scoring="accuracy")
    
    #Fit and store predictions
    dt_class.fit(train_X_acc, train_y)
    predictions_acc_train[scenario] = dt_class.predict(train_X_acc)
    predictions_acc_test[scenario] = dt_class.predict(test_X_acc)
    
    #Calulate accuracy
    acc_train[scenario] = accuracy_score(train_y,predictions_acc_train[scenario])
    acc_test[scenario] = accuracy_score(test_y,predictions_acc_test[scenario])
    
    if (label !='beide'):
        print("F1 ",label)
        estimator = RandomForestClassifier(n_estimators = 100, verbose=1, n_jobs=-1, random_state=42)
        selector_f1[label] = RFE(estimator, num_attributes_f1[label], step=1, verbose=1)
        selector_f1[label] = selector_f1[label].fit(train_X, train_y)
        #Select attributes for F1
        supp_f1 = selector_f1[label].get_support() 
        sel_feat_f1 = train_X.columns[(supp_f1)]
        sel_feat_f1 = sel_feat_f1.tolist()
        print(len(sel_feat_f1))
        print(sel_feat_f1)
        train_X_f1 = copy.deepcopy(train_X[sel_feat_f1])
        test_X_f1 = copy.deepcopy(test_X[sel_feat_f1])


        print("AUC ",label)
        estimator = RandomForestClassifier(n_estimators = 100, verbose=1, n_jobs=-1, random_state=42)
        selector_auc[label] = RFE(estimator, num_attributes_auc[label], step=1, verbose=1)
        selector_auc[label] = selector_auc[label].fit(train_X, train_y)
        #Select attributes for ROC_AUC
        supp_auc = selector_auc[label].get_support() 
        sel_feat_auc = train_X.columns[(supp_auc)]
        sel_feat_auc = sel_feat_auc.tolist()
        print(len(sel_feat_auc))
        print(sel_feat_auc)
        train_X_auc = copy.deepcopy(train_X[sel_feat_auc])
        test_X_auc = copy.deepcopy(test_X[sel_feat_auc])
    
        #Cross_Validation on train data
        func.cross_val(dt_class, train_X_f1, train_y, scoring="f1")
        func.cross_val(dt_class, train_X_auc, train_y, scoring="roc_auc")
        
        dt_class.fit(train_X_f1, train_y)
        predictions_f1_train[scenario] = dt_class.predict(train_X_f1)
        predictions_f1_test[scenario] = dt_class.predict(test_X_f1)

        dt_class.fit(train_X_auc, train_y)
        predictions_auc_train[scenario] = dt_class.predict(train_X_auc)
        predictions_auc_test[scenario] = dt_class.predict(test_X_auc)

        #Calculate performance
        f1_train[scenario] = f1_score(train_y,predictions_f1_train[scenario])
        f1_test[scenario] = f1_score(test_y,predictions_f1_test[scenario]) 
        probs_class_train[scenario] = dt_class.predict_proba(train_X_auc)
        probs_class_train[scenario]  = probs_class_train[scenario][:, 1]
        probs_class_test[scenario] = dt_class.predict_proba(test_X_auc)
        probs_class_test[scenario]  = probs_class_test[scenario][:, 1]
        auc_train[scenario] = roc_auc_score(train_y,probs_class_train[scenario])
        auc_test[scenario] = roc_auc_score(test_y,probs_class_test[scenario]) 

    scenario += 1
    other_count += 1

print("Accuracy Trainingsdaten:", acc_train)
print("Accuracy Testdaten:",acc_test)
print("F1 Trainingsdaten:", f1_train)
print("F1 Testdaten:",f1_test)
print("AUC Trainingsdaten:", auc_train)
print("AUC Testdaten:",auc_test)

Accuracy  Prog1_beste
Fitting estimator with 84 features.


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.2s finished


Fitting estimator with 83 features.


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.1s finished


Fitting estimator with 82 features.


...

Fitting estimator with 61 features.


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.1s finished


Fitting estimator with 60 features.


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.1s finished


59
['AbiBL', 'Abinote', 'BFI_K_11', 'BFI_K_12', 'BFI_K_14', 'BFI_K_17', 'BFI_K_19', 'BFI_K_2', 'BFI_K_6', 'BFI_K_7', 'BFI_K_9', 'BM_Inf_17', 'BM_Inf_18', 'BM_Inf_8', 'Ktyp_psych', 'Kurs_Inf', 'LMI_1', 'LMI_10', 'LMI_11', 'LMI_12', 'LMI_13', 'LMI_14', 'LMI_15', 'LMI_16', 'LMI_17', 'LMI_18', 'LMI_19', 'LMI_2', 'LMI_20', 'LMI_21', 'LMI_22', 'LMI_23', 'LMI_24', 'LMI_25', 'LMI_28', 'LMI_29', 'LMI_3', 'LMI_4', 'LMI_5', 'LMI_6', 'LMI_7', 'LMI_8', 'LMI_9', 'SKInf_1', 'SKInf_2', 'SKInf_3', 'SKInf_4', 'SKMat_1', 'SKMat_2', 'SKMat_3', 'SKMat_4', 'mean_BFI_K_E', 'mean_BFI_K_G', 'mean_BFI_K_N', 'mean_BFI_K_V', 'mean_LMI', 'mean_SKInf', 'mean_SKMat', 'Std_Inf']
Scores: [0.59090909 0.36363636 0.40909091 0.38095238 0.4       ]
Mean: 0.4289177489177489
Standard deviation: 0.08250196415656402
Accuracy Trainingsdaten: [0.9581151832460733, 1.0, 0.9740932642487047, 0.9685863874345549, 1.0, 0.9844559585492227, 1.0]
Accuracy Testdaten: [0.21686746987951808, 0.23684210526315788, 0.21428571428571427, 0.6746987